# NBA player of the week

Игроки недели с 1984-1985 по 2017-2018 сезон


In [181]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import time
import warnings
%matplotlib inline

### Задача 
На датасете,используя алгоритмы ML,научиться определять,является ли игрок защитником или нападающим

In [182]:
# Загрузим датасет
# https://www.kaggle.com/jacobbaruch/nba-player-of-the-week

dataNBA = pd.read_csv('source/NBA_player_of_the_week.csv', sep = ',', engine = 'python')

In [ ]:
# Выкинем менее информативные столбцы и для большей наглядности поменяем некоторые столбцы местами

dataNBA = dataNBA.drop([
    'Conference',
    'Season short',
    'Real_value',
    'Draft Year'
   ], axis=1, errors='ignore')
dataNBA = dataNBA[['Player','Age','Height','Weight','Position','Seasons in league','Season','Date','Team']]

print(dataNBA.shape) 
print(dataNBA.columns)
dataNBA.head()

 - `Age` – Возраст
 - `Data` – Дата,когда выбран лучшим игроком недели
 - `Draft Year` – Год драфта
 - `Height` – Рост
 - `Player` – Имя игрока 
 - `Position` – Позиция на поле
 - `Season` – Сезон
 - `Seasons in league` – Сезонов в лиге сыграно
 - `Team` – Название команды
 - `Weight` – Вес
					

In [ ]:
# Глянем картинку с позициями на поле :
from IPython.display import Image
Image("img/positions.jpg")

- PG(1)  - Разыгрывающий защитник (англ.  Point Guard), или первый номер
- SG(2)  - Атакующий защитник (англ. Shooting Guard), или второй номер
- SF(3)  - Лёгкий форвард (англ. Small Forward), или третий номер
- PF(4)  - Тяжёлый форвард (англ. Power Forward), или четвёртый номер
- C(5)   - Центровой (англ. Center), или пятый номер

Доп обозначения :
- G   - PG или  SG
- F   - SF или PF
- FC  - PF или C

In [185]:
# Определим некоторые вспомогательные функции,необходимые для приведения веса и роста к нужным единицам измерения

def convert_height(height):
    if len(re.split('cm',height)) == 1:
        height = convert_fais_to_cms(height,'-')
    else:
        height = height[:3]  # обрезаем подстроку 'cm'
    return round(int(height))

def convert_fais_to_cms(fai,delimeter): # fai - feet and inches
    # константы :
    cms_in_one_inch = 2.54
    cms_in_one_foot = 30.48 
    # перевод :
    feet,inches = re.split(delimeter,str(fai),maxsplit=1)  # макс кол-во разбиений = 1
    height_in_cms = int(feet) * cms_in_one_foot + int(inches) * cms_in_one_inch;
    return height_in_cms

def convert_weight(weight):
    lb = 0.45359237  # кг в 1-ом фунте 
    weight = re.split('kg',weight)
    if len(weight) == 1:
        weight = int(weight[0]) * lb 
    else:
        weight = int(weight[0])
    return round(weight)

In [186]:
# Проёдёмся по датасету и приведём вес к "кг",а рост к "см"

for i in range(dataNBA.shape[0]):
    dataNBA.at[i,'Height'] = convert_height(dataNBA.at[i,'Height'])
    dataNBA.at[i,'Weight'] = convert_weight(dataNBA.at[i,'Weight'])

In [ ]:
# Посмотрим первые строки датасета
dataNBA.head()

In [ ]:
# Теперь посмотрим сколько уникальных записей,проверяя по имени игрока
dataNBA.Player.unique().shape

In [ ]:
# Маловато,но что поделать.
# Теперь выкинем повторения и сравним 
dataNBA = dataNBA[~dataNBA.duplicated(subset=['Player'],keep='first')]
dataNBA.shape

In [ ]:
# Дропнем ненужные для дальнейшей работы колонки и посмотрим на текущее состояние датасета
dataNBA = dataNBA.drop([ 'Player',
                         'Season',
                         'Date'  ], axis=1, errors='ignore')
dataN

In [ ]:
# Проверим типы столбцов
dataNBA.dtypes

In [192]:
# Приведём вес и рост к int
dataNBA['Weight'] = dataNBA.Weight.astype(int)
dataNBA['Height'] = dataNBA.Height.astype(int)

In [ ]:
# Проверим типы теперь
dataNBA.dtypes

In [ ]:
dataNBA.describe()

In [ ]:
# Нарисуем графики числовых столбцов друг от друга
from pandas.plotting import scatter_matrix
scatter_matrix(dataNBA, alpha = .05, figsize = (10, 10))
pass

In [ ]:
# Корреляция столбцов
dataNBA.corr()

In [ ]:
# Построим график зав-ти кол-ва сезонов в лиге от возраста

plt.plot(dataNBA['Age'], dataNBA['Seasons in league'], 'o', alpha = 0.05)
plt.xlabel('Age')
plt.ylabel('Seasons in league')
plt.title('Age vs Seasons in league')
pass

In [198]:
# Ближе к нашей задаче :

In [199]:
# Условно поделим игроков на защитников и нападающих
# Для этого сначала уберём игроков с позицией 'GF' или 'G-F' - это универсалы,которые играют(ли) как в защите,
# так и в нападении.

dataNBA = dataNBA[(dataNBA.Position != 'GF') & (dataNBA.Position != 'G-F')] 

In [ ]:
# Теперь все игроков с позициями  'PG','SG','G' "запишем" в защитники,
# а с позициями 'SF','PF','F','C','FC','F-C' в нападающие
# Посмотрим для нападающих и защитников зависимость роста от веса

G = dataNBA[(dataNBA['Position'] == 'PG' ) | 
            (dataNBA['Position'] == 'SG' ) | 
            (dataNBA['Position'] == 'G'  ) ]

F = dataNBA[(dataNBA['Position'] == 'SF' ) |
            (dataNBA['Position'] == 'PF' ) |
            (dataNBA['Position'] == 'F'  ) | 
            (dataNBA['Position'] == 'C'  ) |
            (dataNBA['Position'] == 'FC' ) |
            (dataNBA['Position'] == 'F-C'  )]
            
plt.figure(figsize = (10, 6)) 
plt.scatter(G['Weight'],G['Height'],
            alpha = 0.25,
            label = 'Guards',
            color = 'b')

plt.scatter(F['Weight'],F['Height'],
            alpha = 0.25,
            label = 'Forwards',
            color = 'r')

plt.xlabel('Weight')
plt.ylabel('Height')
plt.legend()
plt.grid()

In [ ]:
dataNBA.describe(include = ['object']) # Признаки с типом object 

In [ ]:
dataNBA.describe(include = 'all') # Все признаки

In [ ]:
dataNBA['Team'].unique()  # Уникальные команды

# Задача

In [ ]:
# Поделим колонки на категориальные и числовые
categorical_columns = [c for c in dataNBA.columns if dataNBA[c].dtype.name == 'object'] #
numerical_columns   = [c for c in dataNBA.columns if dataNBA[c].dtype.name != 'object']
print('категориальные : ',categorical_columns)
print('числовые : ',numerical_columns)

In [ ]:
for c in categorical_columns:
    print(c, dataNBA[c].unique())

In [ ]:
# Медианы числовых колонок
dataNBA.median(axis = 0)

In [207]:
# Заполним медианами возможные NA значания
dataNBA = dataNBA.fillna(dataNBA.median(axis = 0), axis = 0)

In [ ]:
dataNBA[numerical_columns].count(axis=0)

In [ ]:
dataNBA[categorical_columns].count(axis = 0)

In [ ]:
dataNBA['Position'].unique()

In [ ]:
# Сделаем колонку 'Position' бинарной - защитников обозначим за 0,нападающих за 1

dataNBA.at[(dataNBA['Position'] == 'PG' ) | 
           (dataNBA['Position'] == 'SG' ) | 
           (dataNBA['Position'] == 'G'  ), 'Position'] = 0


dataNBA.at[(dataNBA['Position'] == 'SF' ) |
           (dataNBA['Position'] == 'PF' ) |
           (dataNBA['Position'] == 'F'  ) | 
           (dataNBA['Position'] == 'C'  ) |
           (dataNBA['Position'] == 'FC' ) |
           (dataNBA['Position'] == 'F-C'  ),'Position'] = 1
dataNBA['Position'].describe()

# Векторизация
Нужно что-то сделать с категориальными (номинальными) признаками

In [ ]:
dataNBA_describe = dataNBA.describe(include =[object])
binary_columns = [c for c in categorical_columns if dataNBA_describe[c]['unique']==2]
nonbinary_columns = [c for c in categorical_columns if dataNBA_describe[c]['unique']>2]
print ('Бинарные- ',binary_columns)
print ('Небинарные- ',nonbinary_columns)

In [ ]:
dataNBA_nonbinary = pd.get_dummies(dataNBA[nonbinary_columns])
print(dataNBA_nonbinary.columns)

# Нормализация количественных признаков

In [ ]:
dataNBA_numerical = dataNBA[numerical_columns]
dataNBA_numerical.describe()

In [215]:
dataNBA_numerical = (dataNBA_numerical - dataNBA_numerical.mean(axis = 0))/dataNBA_numerical.std(axis = 0)

In [ ]:
dataNBA_numerical.describe()

In [ ]:
dataNBA = pd.concat((dataNBA_numerical, dataNBA_nonbinary, dataNBA[binary_columns]), axis = 1)
dataNBA.describe()

In [218]:
dataNBA['Position'] = dataNBA.Position.astype(int)

# X и y

In [ ]:
# Выкинем столбец 'Position'
X = dataNBA.drop(('Position'), axis = 1)
y = dataNBA['Position']
feature_names = X.columns
feature_names

In [ ]:
print(X.shape)
print(y.shape)
N, d = X.shape

In [ ]:
type(X)

In [ ]:
type(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 

print(N_train, N_test)

In [ ]:
181 + 78

#  KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 10)
#knn.set_params(n_neighbors=10)
knn.fit(X_train, y_train)

In [ ]:
y_train_predict = knn.predict(X_train)
y_test_predict = knn.predict(X_test)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

print(err_train, err_test)

In [ ]:
err_test = 1 - knn.score(X_test, y_test)
err_test

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_test_predict))

In [ ]:
confusion_matrix([0,1,0,1,0], [0,1,0,0,0])

### Подбор оптимального кол-ва соседей для метода

In [ ]:
from sklearn.model_selection import GridSearchCV
nnb = [1, 3, 5, 10, 15, 20, 25, 35, 45, 55]
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid = {'n_neighbors': nnb}, cv=10)
grid.fit(X_train, y_train)

best_cv_err = 1 - grid.best_score_
best_n_neighbors = grid.best_estimator_.n_neighbors
print('лучшая ошибка',best_cv_err)
print('лучшее кол-во соседей из представленных - ',best_n_neighbors)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = best_n_neighbors).fit(X_train, y_train)

err_train = np.mean(y_train != knn.predict(X_train))
err_test  = np.mean(y_test  != knn.predict(X_test))

print(err_train, err_test)

Было : 0.153846153846 (при n_neighbors = 10)

Стало : 0.0769230769231 (при n_neighbors = 25)